Michael Perrine
Week 9 Assignment
DSC 550 Data Mining
Professor Werner

<h1><center>Best Model Selection and Hyperparameter Tuning</center></h1>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings

In [2]:
warnings.filterwarnings('ignore')

1. Import the dataset and ensure that it loaded properly.

In [3]:
# Import data and validate by viewing the first five rows
loan = pd.read_csv(r'Loan_Train.csv')
loan.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


2. Prepare the data for modeling by performing the following steps


- Drop the column “Load_ID.”


In [4]:
# This code drops the load id column and displays the result
loan.drop('Loan_ID', axis=1, inplace = True)
loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


- Drop any rows with missing data.


In [5]:
# This code drops the  all NaN values and displays the result
loan.dropna(inplace= True)
loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


- Convert the categorical features into dummy variables.

In [6]:
# This code creates dummy variables
loan_dummy = pd.get_dummies(loan).replace({True : 1, False :0})
loan_dummy.head()


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,1,0,0,1,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,1,1,0,0,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,1


3.Split the data into a training and test set, where the “Loan_Status” column is the target.

In [14]:
#loan['Loan_Status'].replace({'Y' :1 , 'N' :0}, inplace= True)
loan_dummy['Loan_Status_C'] = loan_dummy.apply(lambda row: f"{row['Loan_Status_N']}_{row['Loan_Status_Y']}", axis=1)
loan_dummy.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y,Loan_Status_C
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,1,0,1,0,1,0,0,1,0,1_0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,1,0,0,1,0,0,1,0,1,0_1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,1,1,0,0,0,1,0,1,0_1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,1,0,1,0,0,0,1,0,1,0_1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,1,0,0,1,0,0,1,0,1,0_1


In [16]:
# This code splits data into a training and testing set
X = loan_dummy.drop(columns=['Loan_Status_N', 'Loan_Status_Y', 'Loan_Status_C'], axis=1)
y = loan_dummy['Loan_Status_C']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state= 99)

In [17]:
X_train.shape

(384, 20)

In [18]:
y_train.shape

(384,)

4. Create a pipeline with a min-max scaler and a KNN classifier

In [19]:
# This code creates the min max scaler
scaler = MinMaxScaler()

In [20]:
X_train = scaler.fit_transform(X_train)

In [21]:
X_test = scaler.fit_transform(X_test)

In [22]:
# This code creates the KNN object
knn = KNeighborsClassifier(n_neighbors= 5, n_jobs=-1)

In [23]:
# This code creates the pipeline object
pipe = Pipeline(steps=[('scaler', scaler),('knn', knn)])


5. Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. 

In [24]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

In [25]:
y_pred = pipe.predict(X_test)

In [26]:
ac = accuracy_score(y_test, y_pred)
ac

0.625

6. Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. 

In [27]:
search_space = [{'scale': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

7. Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [28]:
gs = {'n_neighbors': search_space}

In [29]:
knn_gs = KNeighborsClassifier()

In [30]:
knn_param= GridSearchCV(knn_gs, gs, cv=5)

In [31]:
pipe_1 = Pipeline(steps=[('knn_param', knn_param),('knn_gs', knn_gs)])

In [33]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

8. Find the accuracy of the grid search best model on the test set.

9. Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

10. What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

Summary: